In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [ ]:
import warnings
warnings.filterwarnings('ignore')

![alt text](dataset.csv_mindmap.png "Mindmap")

# Préprocessing
| Column name | Column type | Use default transformation ? | Cleaning method | Possible values | Is Label ? |
|------|------|------|------|------|------|
| color | Nominal Qualitative | Yes | Delete Line | YELLOW / PURPLE | No |
| size | Ordinal Qualitative | Yes | Delete Line | SMALL / LARGE | No |
| act | Nominal Qualitative | Yes | Delete Line | STRETCH / DIP | No |
| age | Nominal Qualitative | Yes | Delete Line | ADULT / CHILD | No |
| result | Boolean | Yes | Delete Line | True value = T / False value = F | Yes |


## We import the dataset

In [ ]:
def load_dataset() -> pd.DataFrame: 
	current_dataset = pd.read_csv("dataset.csv", header=None) 
	current_dataset.columns = ['color', 'size', 'act', 'age', 'result'] 
	return current_dataset 

dataframe: pd.DataFrame = load_dataset() 
dataframe

In [ ]:
def clean_dataset(dataframe: pd.DataFrame) -> pd.DataFrame:
	dataframe = dataframe.replace("?", np.nan)
	dataframe = dataframe[dataframe["color"].notna()]
	dataframe = dataframe[dataframe["size"].notna()]
	dataframe = dataframe[dataframe["act"].notna()]
	dataframe = dataframe[dataframe["age"].notna()]
	dataframe = dataframe[dataframe["result"].notna()]
	return dataframe.reset_index(drop=True)

dataframe: pd.DataFrame = load_dataset()

cleaned_dataframe = clean_dataset(dataframe)
cleaned_dataframe


# Splitting

In [ ]:
def split_data(dataframe: pd.DataFrame):
	y = dataframe["result"]
	X = dataframe.drop("result", axis=1)
	
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
	
	X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
	
	return X_train.reset_index(drop=True), y_train.reset_index(drop=True), X_val.reset_index(drop=True), y_val.reset_index(drop=True), X_test.reset_index(drop=True), y_test.reset_index(drop=True)

dataframe: pd.DataFrame = load_dataset()

cleaned_dataframe = clean_dataset(dataframe)

X_train, y_train, X_val, y_val, X_test, y_test = split_data(cleaned_dataframe)
X_train, y_train, X_val, y_val, X_test, y_test

# Transformation

In [ ]:
def transform_data(X_train, X_val, X_test):
	one_hot_encoder = OneHotEncoder( categories=[['YELLOW', 'PURPLE']])
	X_train_encoded_values = one_hot_encoder.fit_transform(X_train[["color"]]).toarray()
	X_val_encoded_values = one_hot_encoder.fit_transform(X_val[["color"]]).toarray()
	X_test_encoded_values = one_hot_encoder.fit_transform(X_test[["color"]]).toarray()
	
	
	X_train_encoded = pd.DataFrame(X_train_encoded_values, columns=[f"color_{x}" for x in one_hot_encoder.categories_[0]])
	X_val_encoded = pd.DataFrame(X_val_encoded_values, columns=[f"color_{x}" for x in one_hot_encoder.categories_[0]])
	X_test_encoded = pd.DataFrame(X_test_encoded_values, columns=[f"color_{x}" for x in one_hot_encoder.categories_[0]])
	X_train = X_train.join(X_train_encoded)
	X_val = X_val.join(X_val_encoded)
	X_test = X_test.join(X_test_encoded)
	
	X_train = X_train.drop("color", axis=1)
	X_val = X_val.drop("color", axis=1)
	X_test = X_test.drop("color", axis=1)
	ordinal_encoder = OrdinalEncoder(categories=[['SMALL', 'LARGE']])
	X_train["size"] = ordinal_encoder.fit_transform(X_train[["size"]])
	X_val["size"] = ordinal_encoder.fit_transform(X_val[["size"]])
	X_test["size"] = ordinal_encoder.fit_transform(X_test[["size"]])
	
	one_hot_encoder = OneHotEncoder( categories=[['STRETCH', 'DIP']])
	X_train_encoded_values = one_hot_encoder.fit_transform(X_train[["act"]]).toarray()
	X_val_encoded_values = one_hot_encoder.fit_transform(X_val[["act"]]).toarray()
	X_test_encoded_values = one_hot_encoder.fit_transform(X_test[["act"]]).toarray()
	
	
	X_train_encoded = pd.DataFrame(X_train_encoded_values, columns=[f"act_{x}" for x in one_hot_encoder.categories_[0]])
	X_val_encoded = pd.DataFrame(X_val_encoded_values, columns=[f"act_{x}" for x in one_hot_encoder.categories_[0]])
	X_test_encoded = pd.DataFrame(X_test_encoded_values, columns=[f"act_{x}" for x in one_hot_encoder.categories_[0]])
	X_train = X_train.join(X_train_encoded)
	X_val = X_val.join(X_val_encoded)
	X_test = X_test.join(X_test_encoded)
	
	X_train = X_train.drop("act", axis=1)
	X_val = X_val.drop("act", axis=1)
	X_test = X_test.drop("act", axis=1)
	one_hot_encoder = OneHotEncoder( categories=[['ADULT', 'CHILD']])
	X_train_encoded_values = one_hot_encoder.fit_transform(X_train[["age"]]).toarray()
	X_val_encoded_values = one_hot_encoder.fit_transform(X_val[["age"]]).toarray()
	X_test_encoded_values = one_hot_encoder.fit_transform(X_test[["age"]]).toarray()
	
	
	X_train_encoded = pd.DataFrame(X_train_encoded_values, columns=[f"age_{x}" for x in one_hot_encoder.categories_[0]])
	X_val_encoded = pd.DataFrame(X_val_encoded_values, columns=[f"age_{x}" for x in one_hot_encoder.categories_[0]])
	X_test_encoded = pd.DataFrame(X_test_encoded_values, columns=[f"age_{x}" for x in one_hot_encoder.categories_[0]])
	X_train = X_train.join(X_train_encoded)
	X_val = X_val.join(X_val_encoded)
	X_test = X_test.join(X_test_encoded)
	
	X_train = X_train.drop("age", axis=1)
	X_val = X_val.drop("age", axis=1)
	X_test = X_test.drop("age", axis=1)
	return X_train, X_val, X_test

dataframe: pd.DataFrame = load_dataset()
cleaned_dataframe = clean_dataset(dataframe)
X_train, y_train, X_val, y_val, X_test, y_test = split_data(cleaned_dataframe)
X_train, X_val, X_test = transform_data(X_train, X_val, X_test)
X_train


# Mining 
| Classifier type | Hyper-parameters | Grid name |
|------|------|------|
| svc_classifier | {'C': [1.0, 2.0]} | grid_search_svc_classifier |
| decision_tree_classifier | {'splitter': ['best', 'random'], 'criterion': ['gini', 'entropy'], 'min_samples_split': [10, 12]} | grid_search_decision_tree_classifier |
| k_neighbor_classifier | {'n_neighbors': [5, 8]} | grid_search_k_neighbor_classifier |
| mlp_classifier | {'solver': ['sgd', 'adam'], 'activation': ['tanh', 'identity']} | grid_search_mlp_classifier |
| random_forest_classifier | {'criterion': ['gini', 'entropy'], 'n_estimators': [5, 2, 7]} | grid_search_random_forest_classifier |


In [ ]:
svc_classifier = SVC()

split = [-1 if i < len(X_train) else 0 for i in range(X_train.shape[0] + X_val.shape[0])]
grid_search_svc_classifier = GridSearchCV(estimator=svc_classifier, param_grid={'C': [1.0, 2.0]}, scoring="accuracy", cv=PredefinedSplit(split), verbose=2)
grid_search_svc_classifier.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
grid_search_svc_classifier.best_params_

In [ ]:
decision_tree_classifier = DecisionTreeClassifier()

split = [-1 if i < len(X_train) else 0 for i in range(X_train.shape[0] + X_val.shape[0])]
grid_search_decision_tree_classifier = GridSearchCV(estimator=decision_tree_classifier, param_grid={'splitter': ['best', 'random'], 'criterion': ['gini', 'entropy'], 'min_samples_split': [10, 12]}, scoring="accuracy", cv=PredefinedSplit(split), verbose=2)
grid_search_decision_tree_classifier.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
grid_search_decision_tree_classifier.best_params_

In [ ]:
k_neighbor_classifier = KNeighborsClassifier()

split = [-1 if i < len(X_train) else 0 for i in range(X_train.shape[0] + X_val.shape[0])]
grid_search_k_neighbor_classifier = GridSearchCV(estimator=k_neighbor_classifier, param_grid={'n_neighbors': [5, 8]}, scoring="accuracy", cv=PredefinedSplit(split), verbose=2)
grid_search_k_neighbor_classifier.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
grid_search_k_neighbor_classifier.best_params_

In [ ]:
mlp_classifier = MLPClassifier()

split = [-1 if i < len(X_train) else 0 for i in range(X_train.shape[0] + X_val.shape[0])]
grid_search_mlp_classifier = GridSearchCV(estimator=mlp_classifier, param_grid={'solver': ['sgd', 'adam'], 'activation': ['tanh', 'identity']}, scoring="accuracy", cv=PredefinedSplit(split), verbose=2)
grid_search_mlp_classifier.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
grid_search_mlp_classifier.best_params_

In [ ]:
random_forest_classifier = RandomForestClassifier()

split = [-1 if i < len(X_train) else 0 for i in range(X_train.shape[0] + X_val.shape[0])]
grid_search_random_forest_classifier = GridSearchCV(estimator=random_forest_classifier, param_grid={'criterion': ['gini', 'entropy'], 'n_estimators': [5, 2, 7]}, scoring="accuracy", cv=PredefinedSplit(split), verbose=2)
grid_search_random_forest_classifier.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
grid_search_random_forest_classifier.best_params_

# Comparaison

In [ ]:
def comparaison_chart(grid_search_name):
	results = grid_search_name.cv_results_
	params = results["params"]
	mean_tst_scores = results["mean_test_score"]
	for index, p in enumerate(params):
		p["mean_test_score"] = mean_tst_scores[index]
	results_dataframe = pd.DataFrame(params)
	
	columns_name = results_dataframe.columns.to_list()
	results_dataframe = results_dataframe.sort_values(by=["mean_test_score"], ascending=False)
	columns_name.remove("mean_test_score")
	results_dataframe["combinaison_hyperparameters"] = results_dataframe[columns_name].apply(lambda x: " | ".join(map(str, x)), axis=1)
	
	
	plt.barh(results_dataframe["combinaison_hyperparameters"], results_dataframe["mean_test_score"])
	plt.xlabel = "score"
	plt.ylabel = "combinaison de paramètres"
	plt.title = "Résultats d entrainement"
	plt.plot()


In [ ]:
comparaison_chart(grid_search_svc_classifier)


In [ ]:
comparaison_chart(grid_search_decision_tree_classifier)


In [ ]:
comparaison_chart(grid_search_k_neighbor_classifier)


In [ ]:
scores = {}
y_test_format = y_test.to_numpy()
y_test_format = np.array(np.where(y_test_format == "no", 0, 1), dtype=int)
prediction = grid_search_svc_classifier.predict(X_test)
prediction = np.array(np.where(prediction == "no", 0, 1), dtype=int)
scores["svc_classifier"] = {}

scores["svc_classifier"]["f1_score"] = f1_score(y_test_format, prediction)

prediction = grid_search_k_neighbor_classifier.predict(X_test)
prediction = np.array(np.where(prediction == "no", 0, 1), dtype=int)
scores["k_neighbor_classifier"] = {}

scores["k_neighbor_classifier"]["f1_score"] = f1_score(y_test_format, prediction)

prediction = grid_search_decision_tree_classifier.predict(X_test)
prediction = np.array(np.where(prediction == "no", 0, 1), dtype=int)
scores["decision_tree_classifier"] = {}

scores["decision_tree_classifier"]["f1_score"] = f1_score(y_test_format, prediction)

plot_df = pd.DataFrame(scores)
for cls in ['svc_classifier', 'k_neighbor_classifier', 'decision_tree_classifier']:
	plot_df.plot(y=cls, kind="bar", use_index=True)
plot_df
